This file is based on implementation of [wsgisler/ibm-ponder-challenge on GitHub](https://github.com/wsgisler/ibm-ponder-challenge/blob/master/2020-03/Ponder%20This%20-%20March%202020%20-%20Challenge.ipynb)

---

In [1]:
%pip install mip

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 2.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
  Created wheel for cffi: filename=cffi-1.15.0-cp312-cp312-linux_x86_64.whl size=479966 sha256=e396f9ae2cb22c7f9ade55a9a3aa226042cf899fd461fa7b12bc876c9c3aba24
  Stored in directory: /home/ptsgrn/.cache/pip/wheels/b9/d6/15/0950847bf7d74ea5f0380b8b23a1d81b45bdf48488b4b8237a
Successfully built cffi
Note: you may need to restart the kernel to use updated packages.


In [1]:
from mip import Model, xsum, BINARY

# Board dimensions
n = 10

# Create a new model
m = Model("dic-dac-doe")

# Variables: X[i][j], O[i][j], and P[i][j] are binary variables indicating
# whether cell (i, j) contains 'X', 'O', or '+', respectively.
# X = [[m.add_var(var_type=BINARY) for j in range(n)] for i in range(n)]
# O = [[m.add_var(var_type=BINARY) for j in range(n)] for i in range(n)]
# P = [[m.add_var(var_type=BINARY) for j in range(n)] for i in range(n)]
# B = [[m.add_var(var_type=BINARY) for j in range(n)] for i in range(n)]

players = [0, 1, 2]
player_symbol = ['x', 'o', '+']
board_size = 10
rows = range(1, board_size+1)
cols = range(1, board_size+1)

choice = {(r, c, p): m.add_var(var_type=BINARY)
          for r in rows for c in cols for p in players}
empty = {(r, c): m.add_var(var_type=BINARY) for r in rows for c in cols}

for r in rows:
    for c in cols:
        m += (xsum(choice[(r, c, p)] for p in players) <= 1)

for p in players:
    m += xsum(choice[r, c, p] for r in rows for c in cols) == (27 if p <= 1 else 26)

for r in rows:
    for c in cols:
        m += xsum(choice[(r, c,p)] for p in players) + empty[(r,c)] == 1

In [11]:
from mip import Model, BINARY, xsum

# Board dimensions
n = 10

# Create a new model
m = Model()

# Variables: X[i][j], O[i][j], and P[i][j] are binary variables indicating
# whether cell (i, j) is occupied by 'X', 'O', '+', or empty respectively.
X = [[m.add_var(var_type=BINARY) for j in range(n)] for i in range(n)]
O = [[m.add_var(var_type=BINARY) for j in range(n)] for i in range(n)]
P = [[m.add_var(var_type=BINARY) for j in range(n)] for i in range(n)]
E = [[m.add_var(var_type=BINARY) for j in range(n)] for i in range(n)]

# Constraint 1: Each cell can be occupied by at most one symbol
for i in range(n):
    for j in range(n):
        m += X[i][j] + O[i][j] + P[i][j] + E[i][j] == 1

# Constraint 2: Total number of occupied cells should be exactly 80
m += xsum(X[i][j] + O[i][j] + P[i][j]
          for i in range(n) for j in range(n)) == 80

# Constraint 3: Prevent three consecutive symbols horizontally, vertically, or diagonally
for i in range(n):
    for j in range(n):
        if j <= n - 3:
            # Horizontal triplets
            m += xsum(X[i][j+k] for k in range(3)) <= 2
            m += xsum(O[i][j+k] for k in range(3)) <= 2
            m += xsum(P[i][j+k] for k in range(3)) <= 2
            m += xsum(E[i][j+k] for k in range(3)) <= 2
        if i <= n - 3:
            # Vertical triplets
            m += xsum(X[i+k][j] for k in range(3)) <= 2
            m += xsum(O[i+k][j] for k in range(3)) <= 2
            m += xsum(P[i+k][j] for k in range(3)) <= 2
            m += xsum(E[i+k][j] for k in range(3)) <= 2
        if i <= n - 3 and j <= n - 3:
            # Diagonal (top-left to bottom-right)
            m += xsum(X[i+k][j+k] for k in range(3)) <= 2
            m += xsum(O[i+k][j+k] for k in range(3)) <= 2
            m += xsum(P[i+k][j+k] for k in range(3)) <= 2
            m += xsum(E[i+k][j+k] for k in range(3)) <= 2
        if i >= 2 and j <= n - 3:
            # Diagonal (bottom-left to top-right)
            m += xsum(X[i-k][j+k] for k in range(3)) <= 2
            m += xsum(O[i-k][j+k] for k in range(3)) <= 2
            m += xsum(P[i-k][j+k] for k in range(3)) <= 2
            m += xsum(E[i-k][j+k] for k in range(3)) <= 2

# Objective function (arbitrary as we're seeking feasibility)
m.objective = xsum(0 for i in range(n) for j in range(n))

# Optimize the model
m.optimize()

# Check if a feasible solution was found
print(P)
if m.num_solutions:
    board = [['.' for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if X[i][j].x >= 0.99:
                board[i][j] = 'X'
            elif O[i][j].x >= 0.99:
                board[i][j] = 'O'
            elif P[i][j].x >= 0.99:
                board[i][j] = '+'
    # Print the resulting board
    for row in board:
        print(' '.join(row))
else:
    print("No feasible solution found.")

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 1253 (0) rows, 400 (0) columns and 4156 (0) elements
Clp1000I sum of infeasibilities 3.49585e-10 - average 2.78998e-13, 8 fixed columns
Coin0506I Presolve 1216 (-37) rows, 392 (-8) columns and 4029 (-127) elements
Clp0006I 0  Obj 0
Clp0029I End of values pass after 392 iterations
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 0
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Clp0032I Optimal objective 0 - 0 iterations time 0.072, Idiot 0.07

Starting MIP optimization
Cgl0004I processed model has 1253 rows, 400 columns (400 integer (400 of which binary)) and 4156 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 0.312%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominat